### Import Packages

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import datetime
from pandas.io.json import json_normalize

In [2]:
startDate = "2017-01-01"
datelist = pd.date_range(pd.to_datetime(startDate), periods=366).tolist() #1 year of data

In [3]:
requestsDataFrame = pd.DataFrame()

for index in range(len(datelist)-1):
    day = str(datelist[index].date())
    nextDay = str(datelist[index+1].date())
    url = """https://secure.toronto.ca/webwizard/ws/requests.json?start_date=""" + day + """T00:00:00Z&end_date=""" + nextDay + """T00:00:00Z&jurisdiction_id=toronto.ca"""
    response = requests.get(url)
    response = json_normalize(response.json()['service_requests'])
    assert response.shape[0] < 1000
    requestsDataFrame = pd.concat([requestsDataFrame, response])

In [38]:
requestsDataFrame.to_pickle("./rawData.pkl")

### Exploratory Data Analysis

In [42]:
assert len(pd.to_datetime(requestsDataFrame.requested_datetime).dt.date.unique()) == 365

365